In [5]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Owner\OneDrive\Pictures\Documents\DataAnalystPortfolio\NetFlix.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s10,Movie,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",India,15-Dec-17,2008,TV-MA,143,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...
2,s100,Movie,3 Heroines,Iman Brotoseno,"Reza Rahadian, Bunga Citra Lestari, Tara Basro...",Indonesia,05-Jan-19,2016,TV-PG,124,"Dramas, International Movies, Sports Movies",Three Indonesian women break records by becomi...
3,s1000,Movie,Blue Mountain State: The Rise of Thadland,Lev L. Spiro,"Alan Ritchson, Darin Brooks, James Cade, Rob R...",United States,01-Mar-16,2016,R,90,Comedies,New NFL star Thad buys his old teammates' belo...
4,s1001,TV Show,Blue Planet II,NaN,David Attenborough,United Kingdom,03-Dec-18,2017,TV-G,1,"British TV Shows, Docuseries, Science & Nature TV",This sequel to the award-winning nature series...


In [7]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4808 entries, 1 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       4808 non-null   object
 1   type          4808 non-null   object
 2   title         4808 non-null   object
 3   director      4808 non-null   object
 4   cast          4808 non-null   object
 5   country       4808 non-null   object
 6   date_added    4808 non-null   object
 7   release_year  4808 non-null   int64 
 8   rating        4808 non-null   object
 9   duration      4808 non-null   int64 
 10  genres        4808 non-null   object
 11  description   4808 non-null   object
dtypes: int64(2), object(10)
memory usage: 488.3+ KB


In [8]:
data = df.to_dict('records') # Get only 1000 records. More records will make it slower to index
len(data)

4808

In [9]:
data[:5]

[{'show_id': 's10',
  'type': 'Movie',
  'title': '1920',
  'director': 'Vikram Bhatt',
  'cast': 'Rajneesh Duggal, Adah Sharma, Indraneil Sengupta, Anjori Alagh, Rajendranath Zutshi, Vipin Sharma, Amin Hajee, Shri Vallabh Vyas',
  'country': 'India',
  'date_added': '15-Dec-17',
  'release_year': 2008,
  'rating': 'TV-MA',
  'duration': 143,
  'genres': 'Horror Movies, International Movies, Thrillers',
  'description': 'An architect and his wife move into a castle that is slated to become a luxury hotel. But something inside is determined to stop the renovation.'},
 {'show_id': 's100',
  'type': 'Movie',
  'title': '3 Heroines',
  'director': 'Iman Brotoseno',
  'cast': 'Reza Rahadian, Bunga Citra Lestari, Tara Basro, Chelsea Islan',
  'country': 'Indonesia',
  'date_added': '05-Jan-19',
  'release_year': 2016,
  'rating': 'TV-PG',
  'duration': 124,
  'genres': 'Dramas, International Movies, Sports Movies',
  'description': 'Three Indonesian women break records by becoming the first 

In [15]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [16]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Owner\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owner\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [38]:
# Create collection to store movies
qdrant.recreate_collection(
    collection_name="NetFlix",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [39]:
# vectorize!
qdrant.upload_points(
    collection_name="NetFlix",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)


In [62]:
user_prompt = "Suggest me a few fantasy movies about dragons "

In [63]:
# Search time for movies!
hits = qdrant.search(
    collection_name="NetFlix",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'show_id': 's1856', 'type': 'Movie', 'title': 'Dragons: Dawn of the Dragon Racers', 'director': 'Elaine Bogan, John Sanford', 'cast': 'Jay Baruchel, America Ferrera, Zack Pearlman, Christopher Mintz-Plasse, T.J. Miller, Andree Vermeulen', 'country': 'United States', 'date_added': '16-Dec-18', 'release_year': 2014, 'rating': 'TV-PG', 'duration': 27, 'genres': 'Children & Family Movies, Comedies', 'description': 'The "How to Train Your Dragon" adventure continues when a hunt for lost sheep turns into a fierce, fun-filled competition.'} score: 0.45616815333882416
{'show_id': 's1852', 'type': 'Movie', 'title': 'Dragonheart: A New Beginning', 'director': 'Doug Lefler', 'cast': 'Christopher Masterson, Harry Van Gorkum, Rona Figueroa, Matt Hickey, Henry O, Tom Burke, Robby Benson, Ken Shorter, John Woodnutt, Lubomir Misak', 'country': 'United States', 'date_added': '01-Jan-20', 'release_year': 2000, 'rating': 'PG', 'duration': 84, 'genres': 'Action & Adventure, Sci-Fi & Fantasy', 'descriptio

In [64]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [65]:
import openai

# Setup the OpenAI client with your API key
client = openai.OpenAI(
    api_key="API_key"  # Replace this with your actual OpenAI API key
)

# Prepare your chat messages
messages = [
    {"role": "system", "content": "You are chatbot, a netflix specialist. Your top priority is to help guide users into selecting netflix movies and telling them a brief description."},
    {"role": "user", "content": "Suggest me an adventure movie with outlaws"},
    {"role": "assistant", "content": str(search_results)}
]

# Call the API to get a chat completion
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",  # Choose the appropriate model. For chat, "gpt-3.5-turbo" is suitable
    messages=messages,
    max_tokens=150
)

# Print the response from the model
print(completion.choices[0].message)


ChatCompletionMessage(content='I recommend "Dragonheart" - In ancient times when majestic fire-breathers soared through the skies, a knight comes face to face and heart to heart with the last dragon on Earth. It combines adventure with mythical creatures and outlaws.', role='assistant', function_call=None, tool_calls=None)
